# imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
import os
from keras import optimizers
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers import merge, GlobalAveragePooling2D
from keras.layers.merge import Add, Multiply, Average, Maximum, Concatenate, Dot
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras import backend as K

from keras.optimizers import Adam

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

data_dir = '/home/ubuntu/data/iceberg'

import keras
print(keras.__version__)

Using TensorFlow backend.


2.1.1


# data load and exclude the rows where the inc angle is na

In [2]:
data_dir = '/home/ubuntu/data/iceberg'

def load_data(data_dir):
    train = pd.read_json(os.path.join(data_dir, 'train.json'))
    test = pd.read_json(os.path.join(data_dir, 'test.json'))
    # Fill 'na' angles with zero
    train.inc_angle = train.inc_angle.replace('na', 0)
    train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
    test.inc_angle = test.inc_angle.replace('na', 0)
    test.inc_angle = test.inc_angle.astype(float).fillna(0.0)
    return train, test

train, test = load_data(data_dir)
print(train.shape, test.shape)

(1604, 5) (8424, 4)


In [3]:
from IPython.display import display

zeros = train[train['inc_angle']==0.0000]
print(zeros.shape)

train_no_zeros = train[train['inc_angle']!=0.0000]
print(train_no_zeros.shape)

# create df without the 0 inc_angles

pd.set_option('display.max_rows', 2000)
display(train_no_zeros)

(133, 5)
(1471, 5)


,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0
5,"[-20.769371, -20.769434, -25.906025, -25.90602...","[-29.288746, -29.712593, -28.884804, -28.88480...",b51d18b5,36.9034,1
6,"[-26.673811, -23.666162, -27.622442, -28.31768...","[-24.557735, -26.97868, -27.622442, -29.073456...",31da1a04,34.4751,1
7,"[-24.989119, -27.755224, -25.817074, -24.98927...","[-27.755173, -26.732174, -28.124943, -31.83772...",56929c16,41.1769,0
8,"[-17.146641, -17.146572, -17.994583, -19.44553...","[-25.733608, -24.472507, -24.710424, -22.77215...",525ab75c,35.7829,0
9,"[-24.020853, -23.551275, -27.18819, -29.126434...","[-28.702518, -33.563324, -29.571918, -29.12643...",192f56eb,43.3007,0


In [4]:
# Process data into images
def process_images(df):
    X_band1 = np.array([np.array(img).astype(np.float32).reshape(75, 75) for img in df["band_1"]])
    X_band2 = np.array([np.array(img).astype(np.float32).reshape(75, 75) for img in df["band_2"]])
    X_3 = (X_band1+X_band2)/2
    
    a = (X_band1 - X_band1.mean()) / (X_band1.max() - X_band1.min())
    b = (X_band2 - X_band2.mean()) / (X_band2.max() - X_band2.min())
    c = (X_3 - X_3.mean()) / (X_3.max() - X_3.min())
    
    # Merge bands and add another band as the mean of Band 1 and Band 2 (useful for the ImageDataGenerator later)
    
    
    imgs = np.concatenate([a[:, :, :, np.newaxis], b[:, :, :, np.newaxis], c[:, :, :, np.newaxis]], axis=-1)
    
    return imgs

X_train_full = process_images(train_no_zeros)
X_test = process_images(test)

X_angle_train_full = np.array(train_no_zeros.inc_angle)
X_angle_test = np.array(test.inc_angle)
y_train_full = np.array(train_no_zeros["is_iceberg"])

In [89]:
def get_scaled_imgs(df):
    imgs = []

    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

#         band_1 *= 255.0/band_1.max()
#         band_2 *= 255.0/band_2.max()
#         band_3 *= 255.0/band_3.max()
        
        imgs.append(np.dstack((band_1, band_2, band_3)))

    return np.array(imgs)

In [90]:
X_train_full = get_scaled_imgs(train_no_zeros)
X_test = get_scaled_imgs(test)

In [91]:
X_train_full[1, :, : ,0].min(), X_train_full[1, :, : ,0].max()


(-23.125309000000001, 12.570409)

In [5]:
# Create a train and validation split, 75% of data used in training
from sklearn.model_selection import train_test_split

X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train_full,
                                    X_angle_train_full, y_train_full, random_state=666, train_size=0.9, test_size=0.1)

X_train.shape, X_valid.shape, X_angle_train.shape, X_angle_valid.shape, y_train.shape, y_valid.shape

((1323, 75, 75, 3), (148, 75, 75, 3), (1323,), (148,), (1323,), (148,))

In [8]:
p = 0
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
# x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
# x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
# x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Conv2D(num_classes,(3,3), padding='same')(x)
x = Dropout(p)(x)
x = GlobalAveragePooling2D()(x)
m = Add()([inc_angle_input, x])
out = Activation('softmax')(m)

optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model_with_inc_angle = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model_with_inc_angle.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model_with_inc_angle.fit([X_train, X_angle_train], to_categorical(y_train),
          batch_size=256,
          epochs=5,
          validation_data=([X_valid, X_angle_valid], to_categorical(y_valid)))

Train on 1323 samples, validate on 148 samples
Epoch 1/5
1323/1323 [==============================] - 7s 5ms/step - loss: 0.7592 - acc: 0.5011 - val_loss: 0.6927 - val_acc: 0.5135
Epoch 2/5
1323/1323 [==============================] - 4s 3ms/step - loss: 0.6378 - acc: 0.6478 - val_loss: 0.6927 - val_acc: 0.5135
Epoch 3/5
1323/1323 [==============================] - 4s 3ms/step - loss: 0.6054 - acc: 0.6939 - val_loss: 0.6931 - val_acc: 0.5135
Epoch 4/5
1323/1323 [==============================] - 4s 3ms/step - loss: 0.5810 - acc: 0.7045 - val_loss: 0.6947 - val_acc: 0.5135
Epoch 5/5
1323/1323 [==============================] - 4s 3ms/step - loss: 0.5648 - acc: 0.7135 - val_loss: 0.6975 - val_acc: 0.5135


In [48]:
lr = 0.0001
K.set_value(model_with_inc_angle.optimizer.lr, lr)

In [18]:
model_with_inc_angle.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 75, 75, 32)   896         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 37, 37, 32)   0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 37, 37, 64)   18496       max_pooling2d_2[0][0]            
__________________________________________________________________________________________________
conv2d_9 (

In [16]:
np.expand_dims(X_angle_train, axis=1).shape

(1103, 1)

In [19]:
X_train.shape

(1103, 75, 75, 3)

In [60]:
X_train


array([[[[-26.75965118, -28.21066284, -27.48515701],
         [-24.42958069, -26.43188477, -25.43073273],
         [-24.95619965, -27.45497513, -26.20558739],
         ..., 
         [-26.76252174, -30.97958755, -28.8710556 ],
         [-23.24073601, -29.04142761, -26.14108276],
         [-25.51964569, -27.45784569, -26.48874664]],

        [[-25.23188591, -29.03852081, -27.13520432],
         [-25.23192406, -27.45493317, -26.34342957],
         [-27.45497513, -28.21074486, -27.83285904],
         ..., 
         [-27.10318756, -27.82743263, -27.46530914],
         [-24.95902824, -25.51960182, -25.23931503],
         [-26.11890793, -24.9590683 , -25.53898811]],

        [[-24.68888092, -27.45493317, -26.07190704],
         [-25.23196411, -27.45497513, -26.34346962],
         [-24.17787743, -28.21078491, -26.19433212],
         ..., 
         [-27.82743263, -26.43471146, -27.131073  ],
         [-28.21357536, -25.81406784, -27.01382065],
         [-27.45784569, -26.43479347, -26.94631958